In [16]:
import numpy as np
import sys, os, shutil
import pandas as pd
import collections as cl
import itertools as it
import timeit
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import copy as cp
import pyprojroot as pr

In [17]:
#Change directory
os.chdir(pr.here("./Data"))
os.getcwd()

'C:\\Users\\yche465\\Desktop\\AIM 1\\Codes\\PrEP-Traj-Clustering\\Data'

### Convert raw data to proportion of moving coverage (PMC) time series

In [18]:
#Import raw dataset
data=pd.read_csv("syndata_13k.csv").astype({'ID':'int'})


In [19]:
data.iloc[1:5,:]

,ID,PrEP_Start,PrEP_End
1,20,10/25/2023,01/11/2024
2,20,05/17/2024,06/26/2024
3,93,03/25/2023,10/26/2023
4,102,12/22/2023,05/06/2024


In [20]:
#the data type of the column is text string
print(type(data['PrEP_Start'][1]))
print(type(data['PrEP_Start'][2]))

<class 'str'>
<class 'str'>


In [21]:
#convert the column's data type to date time 
data['PrEP_Start']= pd.to_datetime(data['PrEP_Start'])
data['PrEP_End']= pd.to_datetime(data['PrEP_End'])

In [22]:
#the data type of the column is now changed to time stamp
print(type(data['PrEP_Start'][1]))
print(type(data['PrEP_End'][1]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [23]:
#create PrEP initiation date variable
startdate=data.groupby("ID").agg('min').drop(columns=['PrEP_End'])
startdate=startdate.rename(columns={'PrEP_Start':'Initiation'})
data2=data.merge(startdate, on="ID",how='left')
#rename ID variable
fdata=data2.rename(columns={'ID':'PID'})

In [24]:
#de-identify the date
a=fdata["PrEP_Start"] - fdata["Initiation"] 
fdata['PrEP_start_days']=a.dt.days
a=fdata["PrEP_End"] - fdata["Initiation"] 
fdata['PrEP_end_days']=a.dt.days
fdata=fdata[['PID','PrEP_start_days','PrEP_end_days']]

In [25]:
#Create binary daily intake/coverage matrix (medcover)
id_pool=np.unique(fdata["PID"])
medcover=np.zeros((len(id_pool),365*2))
for k in range(len(id_pool)):
    Rx_start=np.array(fdata.loc[fdata["PID"]==id_pool[k],"PrEP_start_days"])
    Rx_end=np.array(fdata.loc[fdata["PID"]==id_pool[k],"PrEP_end_days"])
    
    for j in range(len(Rx_start)):
        medcover[k][Rx_start[j]:Rx_end[j]+1]=1

In [26]:
#Create PMC matrix: % of the 14 days with medication intake/coverage
start=np.array(range(0,103*7,7))
end=start+14
PMC=np.zeros((len(id_pool),103))
for k in range(len(id_pool)):    
    for i in range(len(start)):
        PMC[k][i]=sum(medcover[k][start[i]:end[i]])/14
#Create binary PrEP sero-protection matrix (0: PMC<0.57, 1: otherwise)  
SeroProtect=np.multiply(PMC>=4/7,1)

In [27]:
#Append subject ID column to the SeroProtect matrix
SeroProtect_indexed=np.column_stack((id_pool,SeroProtect))

In [28]:
#Export PMC matrix
f= open("PMC_13k.txt","w+")
for i in range(len(PMC)):
    for j in range(103):
        if j == 102:
            f.write("%s\n" % (PMC[i][j]))
        else:
            f.write("%s," % (PMC[i][j]))
f.close()

In [29]:
#Export sero-protection matrix
f= open("SeroProtect_13k.txt","w+")
for i in range(len(SeroProtect_indexed)):
    for j in range(104):
        if j == 103:
            f.write("%s\n" % (SeroProtect_indexed[i][j]))
        else:
            f.write("%s," % (SeroProtect_indexed[i][j]))
f.close()